## install required packages
run the following command line in terminal

       pip install -r requirements.txt

## Import packages

In [1]:
import pandas as pd
import numpy as np
import os
import json
import obonet
import inflect
import networkx as nx
import matplotlib.pyplot as plt
import requests
import re
from openai import OpenAI
from pydantic import BaseModel
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()


False

## Define output format

In [2]:
class cellTypeFormat(BaseModel):
       cellType: str

## Read cell clusters file 
file path set to ./Data/all.csv

In [3]:
def read_cluster(path):
      dataframe = pd.read_csv(path)
      print(dataframe['marker'])
      return dataframe 
gene_list = read_cluster('../Data/all.csv')
print(gene_list.iloc[0])

0       MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
1       MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...
2       IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...
3       IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...
4       GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...
                              ...                        
1125    KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...
1126    JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...
1127    TFF3,PKHD1L1,PROX1,NTS,FLT4,RELN,GPR182,STAB2,...
1128    VWF,ACKR1,PLVAP,PECAM1,AQP1,RAMP3,CLEC14A,SOX1...
1129    MSLN,UPK3B,MUC16,CALB2,KLK11,BICDL1,CHAC1,TGM1...
Name: marker, Length: 1130, dtype: object
dataset                                                               Azimuth
tissue                                                                   PBMC
marker                      MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...
manual_annotation                                         Intermediate B cell
manual_C

## Split genelist
split genelist to multiple chunks so that it won't exceed model input size

In [140]:
# Generate the user message for each chunk
def generate_user_message(gene_list:  pd.DataFrame, max_chunk_size=100) -> list:
    # Split the gene_list into chunks
    lung_data = gene_list[gene_list["tissue"] == "Lung"]


    return lung_data['marker'].to_list()

# Generate messages with a tunable chunk size
messages = generate_user_message(gene_list, max_chunk_size=1)  # Adjust max_chunk_size as needed
print(messages)

['EMP2, AGER, GPRC5A, LMO7, RTKN2, ARHGEF26, LAMA3, DST, SPOCK2, SCEL', 'MS4A1, BANK1, LINC00926, CD79A, BLK, CD79B, VPREB3, SP140, CD22, ADAM28', 'KRT15, S100A2, KRT17, KRT5, KRT19, PERP, CLDN1, FHL2, DST, ITGA2', 'S100A9, S100A12, S100A8, VCAN, FCN1, CD300E, CTSS, CD99, CSF3R, METTL9', 'CD2, IL7R, TRAT1, BCL11B, ITK, CAMK4, CD6, LEF1, MALAT1, MAL', 'RSPH1, CAPS, FAM92B, CDHR3, CCDC170, SNTN, C20orf85, ZMYND10, DNAH6, FANK1', 'HLA-DPB1, FCGR2B, HLA-DMB, CLEC10A, CSF2RA, MS4A6A, HLA-DQA1, RGS1, GPR183, FPR3', 'FBLN1, ADH1B, DCN, LUM, C1S, C1R, SLIT2, C3, CFD, ABLIM1', 'CCL21, MMRN1, PROX1, PPFIBP1, TFF3, TBX1, PKHD1L1, IGFBP7, TFPI, TIMP3', 'ACP5, C1QA, C1QB, VSIG4, MARCO, OLR1, APOC1, SLCO2B1, GPD1, ITGAX', 'PRF1, NKG7, KLRF1, CD247, GNLY, PYHIN1, KLRD1, SPON2, FCGR3A, GZMB', 'MZB1, DERL3, SEC11C, XBP1, TNFRSF17, SSR4, POU2AF1, HSP90B1, PRDX4, MANF', 'SFTPA1, SFTPA2, NAPSA, SFTPB, SFTPD, SFTPC, SLC34A2, ABCA3, LAMP3, LPCAT1', 'KIT, CPA3, TPSAB1, HPGDS, VWA5A, MS4A2, HDC, CD44, GATA2, 

## Gemeni ai

### model configuration
set the GOOGLE_API_KEY under .env file

In [5]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
gemeni_model = genai.GenerativeModel("gemini-1.5-flash")
gemeni_config = genai.GenerationConfig(response_mime_type="application/json",response_schema=list[cellTypeFormat])

NameError: name 'genai' is not defined

### Send prompt to gemini

In [ ]:
earlyStop = 2
Prompt = "Identify cell types using the following tissue name and markers separately for each row. Only provide the cell type name. Do not show numbers before the name. Some can be a mixture of multiple cell types."
def annotateCell_Gemini(messages: list)->list:
    global gemeni_model, gemeni_config 
    responses = []
    i = 0
    for message in messages:
       i+=1
       if i> earlyStop:
              break
       prompt = Prompt
       for cluster in message:
              prompt += cluster
              prompt += '\n'
       print(prompt)
       response = gemeni_model.generate_content(
              prompt,
              generation_config=gemeni_config
       )
       responses.append(response.text)
    return responses


In [ ]:
annotateCell_Gemini_results = annotateCell_Gemini(messages)
print(annotateCell_Gemini_results)

Identify cell types using the following tissue name and markers separately for each row. Only provide the cell type name. Do not show numbers before the name. Some can be a mixture of multiple cell types.PBMC : MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LINC01857, RALGPS2, BANK1, CD79B
PBMC : MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, RALGPS2, TNFRSF13C, LINC01781
PBMC : IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, TCL1A, CD79B, YBX3
PBMC : IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13B, POU2AF1, CPNE5, HRASLS2, NT5DC2
PBMC : GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2M, IL32, NKG7
PBMC : TCF7, CD4, CCR7, IL7R, FHIT, LEF1, MAL, NOSIP, LDHB, PIK3IP1
PBMC : MKI67, TOP2A, PCLAF, CENPF, TYMS, NUSAP1, ASPM, PTTG1, TPX2, RRM2
PBMC : IL7R, TMSB10, CD4, ITGB1, LTB, TRAC, AQP3, LDHB, IL32, MAL
PBMC : IL7R, CCL5, FYB1, GZMK, IL32, GZMA, KLRB1, TRAC, LTB, AQP3
PBMC : RTKN2, FOXP3, AC133644.2, CD4, IL2RA, TIGIT, CTLA4, FCRL3, LAIR2, IKZF2

Identify cell types using the following tissue name 

## GPT

### model configuration

In [ ]:
# import os
# os.environ["OPENAI_API_KEY"] = 
client = OpenAI(api_key = api_key)

### upload file

In [6]:
def upload_cell_cluster(filename):
       global client
       client.files.create(
              file=open(filename, "rb"),
              purpose="assistant"
       )
       print(client.files.list())

### Create Embeddings

In [161]:
# To replace the legacy openai.embeddings_utils.get_embedding function
def get_embedding(text, model="text-embedding-3-small"):
   text = str(text).replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding
# 加載 RAG 資料庫
cell_type_db = pd.read_csv("lung_canonical_cell_type_markers_from_literature.csv")
marker_importance_db = pd.read_csv("lung_cluster_cell_type_marker_importance.csv")


In [ ]:
# 只要做一次就好
# 計算資料庫嵌入
cell_type_db["embedding"] = cell_type_db["markers"].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
marker_importance_db["embedding"] = marker_importance_db["marker"].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

# 保存嵌入
cell_type_db["embedding"].to_csv("cell_type_embedding.csv")
marker_importance_db["embedding"].to_csv("marker_importance_embedding.csv")

In [163]:
# 沒做上面的話，直接跑這個
import ast  # 或者用 json

if not os.path.exists("marker_importance_embedding.csv"):
    dfs = [pd.read_csv(f"splitted_data/marker_importance_embedding-{i}.csv") for i in range(1, 13)]
    marker_importance_db["embedding"] = pd.concat(dfs, ignore_index=True)["embedding"]
else:
    marker_importance_db["embedding"] = pd.read_csv("marker_importance_embedding.csv")["embedding"]
# 加載嵌入
cell_type_db["embedding"] = pd.read_csv("cell_type_embedding.csv")["embedding"]
# marker_importance_db["embedding"] = pd.read_csv("marker_importance_embedding.csv")
cell_type_db["embedding"] = cell_type_db["embedding"].apply(lambda x: np.array(ast.literal_eval(x), dtype=float))
marker_importance_db["embedding"] = marker_importance_db["embedding"].apply(lambda x: np.array(ast.literal_eval(x), dtype=float))

print(cell_type_db["embedding"][0])
print(marker_importance_db["embedding"][0])

[-0.0022751   0.00393444  0.02531783 ... -0.02559006 -0.02050835
 -0.00551599]
[ 0.00537706 -0.00582684  0.00831246 ... -0.04004051 -0.00739938
 -0.0117619 ]


### send prompt to GPT

In [170]:
from tqdm import tqdm
# Alternate the lagecy openai.embeddings_utils.search_vectors function
def cosine_similarity(a, b):
    # print(a, b, type(a), type(b), type(a[0]), type(b[0]))
    # print(np.linalg.norm(a), np.linalg.norm(b))
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def annotateCell_GPT(messages: list, n=5) -> list:
    global client
    responses = []

    for message in tqdm(messages):

        # 提取輸入的 marker
        # print(message, type(message))
        input_markers = message.split(", ")
        input_embedding = get_embedding(", ".join(input_markers))
        cell_type_db["similarity"] = cell_type_db["embedding"].apply(lambda x: cosine_similarity(x, input_embedding))
        # marker_importance_db["similarity"] = marker_importance_db["embedding"].apply(lambda x: cosine_similarity(x, input_embedding))

        # 進行檢索
        cell_type_results = (
            cell_type_db.sort_values("similarity", ascending=False)
            .head(n)
        )
        # marker_importance_results = (
        #     marker_importance_db.sort_values("similarity", ascending=False)
        #     .head(n)
        # )

        # 組合上下文
        context = "### Cell Type Database Results ###\n"
        for _,  result in cell_type_results.iterrows():
            
            context += f"Cell Type: {result['cell type']}\nMarkers: {result['markers']}\nBroad Type: {result['broad type']}\n"
            tempdb = marker_importance_db[marker_importance_db["cell_type_formatted"] == result["cell type"]]
            if not tempdb.empty:
                for marker in result['markers'].split(", "):
                    imp = tempdb[tempdb["marker"] == marker]
                    if not imp.empty:
                        context += f"Importance of {marker}: {imp['neg_log10_p'].values[0]}\n"
                    else:
                        context += f"Importance of {marker}: Not Found\n"
            context += "\n"

        # 提示用戶
        message = ("### task: Identify cell types ###\n" + message + '\n'
        + "Answer is not necessary to be in the Database. "
        + "Only provide the cell type name, formatted as: {{cell_type_name}}. "
        + "Do not show numbers before the name. "
        + "Some can be a mixture of multiple cell types.")
        # 提供給 GPT 模型
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Identify cell types of lung tissue using the markers provided. Use the retrieved context for your decision."},
                {"role": "user", "content": f"{context}\n\n{message}"}
            ],
            # response_format=json
        )
        print(completion.choices[0].message.content)
        candi = completion.choices[0].message.content
        candi = re.findall(r"\{\{(.*?)\}\}", candi)
        if candi:
            responses.append(candi[0])
        else:
            responses.append("Not Provided")

    return responses


annotateCell_GPT_results = annotateCell_GPT(messages)
print(annotateCell_GPT_results)

  1%|          | 1/126 [00:01<02:12,  1.06s/it]

{{alveolar epithelial cell}}


  2%|▏         | 2/126 [00:01<02:01,  1.02it/s]

{{B cell}}


  2%|▏         | 3/126 [00:03<02:07,  1.04s/it]

...

{{goblet cell}}


 99%|█████████▉| 125/126 [02:14<00:00,  1.01it/s]

{{intermediate monocyte}}


100%|██████████| 126/126 [02:15<00:00,  1.08s/it]

{{goblet cell}}
['alveolar epithelial cell', 'B cell', 'basal epithelial cell', 'neutrophil', 'T cell', 'ciliated cell', 'dendritic cell', 'fibroblast', 'lymphatic endothelial cell', 'macrophage', 'natural killer cell', 'plasma cell', 'alveolar type II cell', 'mast cell', 'neutrophil', 'endothelial cell', 'cytotoxic T cell', 'club cell/alveolar type II cell', 'fibroblast', 'natural killer t cell', 'proliferating epithelial cells', 'proliferating cell', 'endothelial cell', 'club cell', 'endothelial cell', 'endothelial cell', 'smooth muscle cell', 'artery endothelial cell', 'endothelial cell', 'unknown or mixed lineage', 'cell_type_name', 'undefined_cell_type', 'platelet', 'natural killer t cell', 'airway smooth muscle', 'bronchial epithelial cell', 'club cell', 'neuroendocrine cell', 'serous cell', 'macrophage', 'cytotoxic T cell', 'alveolar type II pneumocyte', 'endothelial cell', 'T cell', 'endothelial cell', 'myofibroblast', 'alveolar epithelial type 1 cell', 'alveolar epithelial typ

In [171]:
lung_data = gene_list[gene_list["tissue"] == "Lung"]
lung_data['RAG_cell_type'] = annotateCell_GPT_results
lung_data.to_csv("RAG_no_verbosity.csv", index=False)

/var/folders/sh/zhklvq891q7c_0dsphlk2_dc0000gn/T/ipykernel_98513/1546951321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lung_data['RAG_cell_type'] = annotateCell_GPT_results


## Scoring system

### result parsing

In [149]:
def json_parsing(annotateCell_results: json)->list:
       # Parse the strings into Python objects
       parsed_results = [json.loads(item.strip()) for item in annotateCell_results]
       # Flatten the results to get a list of all cell types
       cell_types = [entry['cellType'] for result in parsed_results for entry in result]
       return cell_types

In [150]:
cell_types = json_parsing(annotateCell_GPT_results)
print(cell_types)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### CL correspondence
match the cell_types to get CLID from Cell Ontology
search name in graph and return the corresponding clid\
if no matches found, send request to cell ontology

In [97]:
# URL for Cell Ontology (CO) OBO file
CO_URL = 'http://purl.obolibrary.org/obo/CL.obo'
OBO_FILE_PATH = 'oboNet/cl.obo'
# dictionary to store search result
NAME_TO_CLID_DICT = dict()

def load_ontology(url):
    graph = obonet.read_obo(url)
    return graph
def search_ontology(cell_name: str, ontology='cl')->str:
    global NAME_TO_CLID_DICT
    normalized_name = inflector.singular_noun(cell_name.lower()) or cell_name.lower()
    normalized_name = re.sub(r"^\(?\d+\)?\.", "", normalized_name).strip()
    # search in dict for faster access to clid and name
    if normalized_name in NAME_TO_CLID_DICT :
        return NAME_TO_CLID_DICT[normalized_name]
    
    for id, data in graph.nodes(data=True):
        if 'name' in data:
            # Singularize and normalize the graph's 'name'
            label = inflector.singular_noun(data['name'].lower()) or data['name'].lower()
            # Compare normalized names
            if label == normalized_name:
                NAME_TO_CLID_DICT[normalized_name] = (id, label)
                return id, label # Return the clid and label(name) if a match is found
    
    # If no clid name found in graph, send request to ontology api    
    # OLS API URL
    url = f"https://www.ebi.ac.uk/ols/api/search?q={normalized_name}&ontology={ontology}"

    # Make the API request
    response = requests.get(url)
    if response.status_code == 200:
        if len(response.json()['response']['docs'])==0:
          return None, None
        id = response.json()['response']['docs'][0]['obo_id']
        label = response.json()['response']['docs'][0]['label']
        NAME_TO_CLID_DICT[normalized_name] = (id, label)
        return id, label
    else:
        print(f"Error: {response.status_code}")
        return None, None

inflector = inflect.engine()
# Load the Cell Ontology
graph = load_ontology(OBO_FILE_PATH)



In [54]:
clids = []
for cell_type in cell_types:
  clids.append(search_ontology(cell_type))

print(clids)

NameError: name 'cell_types' is not defined

### Calculate distance with two CLID
#### Method 1 : shortest path length between two nodes
calculate the distance between two nodes ( manual CLID and LLM annotated CLID)

In [91]:
def calculate_difference(graph:nx.graph, clid_1:str, clid_2:str)->int:
    try:
        return nx.shortest_path_length(graph, source=clid_1, target=clid_2)
    except:
        return -1
def calculate_difference_name(graph:nx.graph, type_1:str, type_2:str)->int:
    clid_1, label_1 = search_ontology(type_1)
    clid_2, label_2 = search_ontology(type_2)
    return calculate_difference(graph, clid_1, clid_2)


In [95]:
print(calculate_difference(graph, 'CL:0002250', 'CL:0009016'))
print(calculate_difference(graph,  'CL:0009016','CL:0002250'))
# intestinal crypt stem cell 0002250
# intestinal crypt stem cell of large intestine 0009016
# should put the broader type (i.e. LLM annotated ) behind
print(calculate_difference_name(graph, 'T cells', 'Cytotoxic T cells'))
print(calculate_difference_name(graph, 'Cytotoxic T cells', 'T cells'))
print(calculate_difference_name(graph, 'T-helper cells', 'T cells'))

-1
1
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
-1
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
3
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 't-helper cell': ('CL:0000912', 'helper T cell')}
3


{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 'helper T cell': ('CL:0000912', 'helper T cell')}
searching
{'t cell': ('CL:0000084', 't cell'), 'cytotoxic t cell': ('CL:0000910', 'cytotoxic t cell'), 'helper T cell': ('CL:0000912', 'helper T cell')}
3


#### Method 2 : calculate adjusted path-based similarity
assign score according to 'On measuring of SImilarity between tree nodes'

$$ l_a (v_i, v_j) = \frac{l(v_i, lca_{ij})+l(v_j, lca_{ij})}{1 + l(lca_{ij},t)}$$

while score equals $$ s_a(v_i, v_j) = \frac{1}{1+l_a(v_i, v_j)} = \frac{1 + l(lca_{ij},t)}{1+ l(v_i, lca_{ij})+l(v_j, lca_{ij}) + l(lca_{ij},t)}$$

In [14]:
def adjusted_path_based_score(graph, clid_1, clid_2):
       ugraph = graph.to_undirected()
       lca = nx.lowest_common_ancestor(graph, clid_1, clid_2)
       root_node = [x for x in graph.nodes() if graph.out_degree(x)>1 and graph.in_degree(x)==0]
       try:
              l_i_lca = nx.shortest_path_length(ugraph, source = clid_1, target = lca)
              l_j_lca = nx.shortest_path_length(ugraph, source = clid_2, traget = lca)
              l_lca_t = nx.shortest_path_length(ugraph, source = lca, target = root_node)
              return (1+l_lca_t) / (1+l_i_lca + l_j_lca + l_lca_t)
       except:
              return -1

In [ ]:
print(nx.is_directed_acyclic_graph(graph))
degraph= graph.copy()
nx.find_cycle(degraph, orientation='original')

while not nx.is_directed_acyclic_graph(degraph):
       trimmed_egdes = nx.find_cycle(degraph, orientation='original')
       for edge in trimmed_egdes:
              print(graph.nodes[edge[0]]['name'],edge[2],graph.nodes[edge[1]]['name'] )
              dagraph = degraph.remove_edge(edge[0], edge[1])


In [43]:
diff_result=[]
for i in range(len(clids)):
  clid_1 =clids[i] #LLM annotated CLID
  clid_2 =gene_list["manual_CLID"][i] 
  if(clid_1 != None and clid_2!="nan"):# manual annotated CLID
    difference = calculate_difference(graph, clid_2, clid_1)
    diff_result.append(difference)
  else:
    diff_result.append(-1)
  print(clid_1, clid_2)
  # print(difference)
print(diff_result)

CL:0000236 CL:0000818
CL:0000236 CL:0000787
CL:0000236 CL:0000788
CL:0000236 CL:0000980
CL:0000910 CL:0000934
CL:0000084 CL:0000895
None nan
CL:0000084 CL:0000904
CL:0000910 CL:0000905
None CL:0000815
None CL:0000900
CL:0000084 nan
CL:0000084 CL:0000907
CL:0000910 CL:0000913
CL:0000576 nan
CL:0000451 CL:0000990
CL:0000451 CL:0000990
CL:0000236 CL:0000784
CL:0000576 CL:0001054
CL:0000235 nan
[2, 3, 3, 3, -1, 3, -1, 5, -1, -1, -1, -1, 5, -1, -1, 1, 1, -1, 1, -1]


### Assign new scoring system to all.csv

In [39]:
MODELS = ['gpt4aug3', 'gpt4mar23','gpt3.5aug3', 'CellMarker2.0','SingleR', 'ScType']
MANUAL = 'manual'

In [40]:
def assign_score(gene_list: pd.DataFrame):
       global MODELS
       global MANUAL
       global graph
       gene_list_scored = pd.DataFrame()
       gene_list_scored = gene_list.copy()
       for MODEL in MODELS:
              scores = []
              for i in range(len(gene_list)):
                     clid_1 = gene_list[MANUAL+'_CLID'][i] 
                     clid_2 = gene_list[MODEL+'_CLID'][i]
                     scores.append(calculate_difference(graph, clid_1, clid_2))
              gene_list_scored[MODEL+'_aggrement_modified'] = scores
       return gene_list_scored


In [41]:
gene_list_scored = assign_score(gene_list)
print(gene_list_scored)

            dataset       tissue  \
0           Azimuth         PBMC   
1           Azimuth         PBMC   
2           Azimuth         PBMC   
3           Azimuth         PBMC   
4           Azimuth         PBMC   
...             ...          ...   
1125  tabulasapiens  Vasculature   
1126  tabulasapiens  Vasculature   
1127  tabulasapiens  Vasculature   
1128  tabulasapiens  Vasculature   
1129  tabulasapiens  Vasculature   

                                                 marker  \
0     MS4A1, TNFRSF13B, IGHM, IGHD, AIM2, CD79A, LIN...   
1     MS4A1, COCH, AIM2, BANK1, SSPN, CD79A, TEX9, R...   
2     IGHM, IGHD, CD79A, IL4R, MS4A1, CXCR4, BTG1, T...   
3     IGHA2, MZB1, TNFRSF17, DERL3, TXNDC5, TNFRSF13...   
4     GZMH, CD4, FGFBP2, ITGB1, GZMA, CST7, GNLY, B2...   
...                                                 ...   
1125  KLRD1,NKG7,XCL2,CTSW,XCL1,GNLY,GZMA,KLRB1,KLRC...   
1126  JCHAIN,MZB1,DERL3,IGHG2,IGHA2,TNFRSF17,SDC1,FC...   
1127  TFF3,PKHD1L1,PROX1,NTS,FLT4,RE

In [42]:
gene_list_scored.to_csv('Data/all_modified.csv')